<a href="https://colab.research.google.com/github/saadkhi/Side/blob/main/Script2_for_fintune_w_LLaMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Packages & Libraries**

In [ ]:
!pip install faiss-cpu
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 73.4 MB/s eta 0:00:00


# **Dataset Info and finrtuning**

In [ ]:
import json

docs = []
with open("dataset.jsonl", "r") as f:
    for line in f:
        row = json.loads(line)
        text = f"Q: {row['prompt']}\nA: {row['completion']}"
        docs.append(text)

print(len(docs))

11171


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

embedder = SentenceTransformer("all-MiniLM-L6-v2")  # lightweight & fast

doc_embeddings = embedder.encode(docs, convert_to_numpy=True, show_progress_bar=True)

# Build FAISS index
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(doc_embeddings)

print("Index size:", index.ntotal)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/350 [00:00<?, ?it/s]

Index size: 11171


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "microsoft/phi-3-mini-4k-instruct"  # or llama2-chat if you have GPU
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

llm = pipeline("text-generation", model=model, tokenizer=tokenizer)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
def is_database_query(query: str) -> bool:
    keywords = ["SQL","NoSQL","Database","Query","Table","Collection","Document","Key-Value","Graph","Schema","Index","Primary Key","Foreign Key","Join","INNER JOIN","LEFT JOIN","RIGHT JOIN","FULL JOIN","SELECT","INSERT","UPDATE","DELETE","WHERE","GROUP BY","HAVING","ORDER BY","LIMIT","OFFSET","DISTINCT","COUNT","SUM","AVG","MAX","MIN","Aggregation","Subquery","CTE","View","Trigger","Stored Procedure","Transaction","ACID","BASE","Normalization","Denormalization","Partitioning","Sharding","Replication","Consistency","Eventual Consistency","Index Scan","Full Scan","Constraint","UNIQUE","NOT NULL","CHECK","MongoDB","Redis","Neo4j","Cassandra","DynamoDB","find","aggregate","$match","$group","$sort","$limit","$skip","$project","$lookup","$unwind","$push","$pull","$set","$unset","SET","GET","LPUSH","RPOP","SADD","ZADD","HSET","HGET","Node","Relationship","MATCH","CREATE","MERGE","Cypher","Index Maintenance","Query Optimization","Execution Plan","EXPLAIN","Materialized View","Caching","TTL","Full-Text Search","Data Modeling","Connection Pooling","Deadlock","ALTER","DROP","TRUNCATE","CREATE TABLE","CREATE INDEX","CREATE VIEW","COMMIT","ROLLBACK","SAVEPOINT","LOCK","UNLOCK","ANALYZE","VACUUM","CLUSTER","REINDEX","CHECKPOINT","CASCADE","RESTRICT","DEFAULT","GENERATED ALWAYS","SEQUENCE","CURSOR","FETCH","OVER","PARTITION BY","RANK","DENSE_RANK","ROW_NUMBER","NTILE","LAG","LEAD","FIRST_VALUE","LAST_VALUE","PERCENTILE_CONT","PERCENTILE_DISC","WITHIN GROUP","LIKE","ILIKE","SIMILAR TO","REGEXP","COALESCE","NULLIF","CASE","UNION ALL","INTERSECT","EXCEPT","CROSS JOIN","NATURAL JOIN","SELF JOIN","MERGE INTO","UPSERT","EXISTS","NOT EXISTS","ANY","ALL","IN","NOT IN","BETWEEN","CAST","CONVERT","CURRENT_DATE","CURRENT_TIMESTAMP","DATEADD","DATEDIFF","EXTRACT","TO_CHAR","TO_DATE","JSON","JSONB","TSVECTOR","TSQUERY","$geoNear","$geoWithin","$elemMatch","$size","$exists","$type","$regex","$in","$nin","$or","$and","$not","$avg","$sum","$min","$max","HDEL","HINCRBY","LINDEX","LLEN","RPUSH","LPOP","SISMEMBER","SDIFF","SUNION","SINTER","ZSCORE","ZCOUNT","RETURN","DETACH DELETE","OPTIONAL MATCH","UNWIND","CALL","YIELD","Backup","Failover","Read Replica","Write-Ahead Logging","Hot Standby","Cold Backup","Snapshot Isolation","Query Cache","Multi-Tenancy","Data Integrity","Index Fragmentation","Query Timeout","Connection Timeout","Read Consistency","Write Conflict"]
    return any(k in query.lower() for k in keywords)

In [ ]:
def chatbot(query):
    # Step 1: Check domain relevance
    if not is_database_query(query):
        return "I only answer database-related queries."

    # Step 2: Retrieve top matches
    q_embedding = embedder.encode([query], convert_to_numpy=True)
    D, I = index.search(q_embedding, k=3)  # top 3

    # Step 3: Check threshold
    if D[0][0] > 0.6:
        context = "\n".join([docs[i] for i in I[0]])
    else:
        context = ""

    # Step 4: Build prompt
    prompt = f"""
    You are a SQL/NoSQL expert.
    Context: {context if context else "No context found."}
    Question: {query}

    Answer:
    """

    # Run through model
    raw_output = llm(prompt, max_new_tokens=300, do_sample=True)[0]['generated_text']

    # Extract only the part after "Answer:"
    if "Answer:" in raw_output:
        answer = raw_output.split("Answer:", 1)[1].strip()
    else:
        answer = raw_output.strip()

    return answer


In [ ]:
print("💬 Database Chatbot is ready! (type 'exit' to quit)\n")

while True:
    query = input("You: ").strip()
    if query.lower() in ["exit", "quit", "bye"]:
        print("Bot: Goodbye! 👋")
        break

    response = chatbot(query)
    print(f"Bot: {response}\n")

💬 Database Chatbot is ready! (type 'exit' to quit)

You: Select all from employee from employee table who have salary less than 10000
Bot: I only answer database-related queries.

You: give me SQL query Select all from employee from employee table who have salary less than 10000
Bot: I only answer database-related queries.

You: what is sql?
Bot: I only answer database-related queries.

You: exit
Bot: Goodbye! 👋
